In [1]:
using Pkg
Pkg.activate("C:/Users/lisah/Documents/Repos/ma-code")

  Activating project at `C:\Users\lisah\Documents\Repos\ma-code`


In [2]:
using DifferentialEquations, CSV, DataFrames, ForwardDiff, Distributions

In [ ]:
# functions i need from other file
function klausmeier!(du,u,p,t)
    du[1] = -u[1] - u[1] * u[2]^2 + p[1] # water compartment
    du[2] = u[1] * u[2]^2 - p[2] * u[2] # biomass compartment
end

function sol_klausmeier(hprm::Hyperprm)
    u0 = [hprm.w0; hprm.n0]
    p = [hprm.a; hprm.m]
    tspan = (0.0, hprm.M-1)

    prob = ODEProblem(klausmeier!, u0, tspan, p)
    sol = solve(prob,
        saveat=1.0  # consider specific time points
    )

    return DataFrame(time=sol.t, w=sol[1, :], n=sol[2, :])
end

function create_grid()
    a_vals = 0.0:0.1:2.0
    n_vals = 0.0:0.1:4.0

    grid = [(a, n) for n in n_vals, a in a_vals] 
    return grid
end

function randomize_data(df::DataFrame, noise::Float64)
    if noise == 0.0
        return df
    else
        df[!, "w"] .= df[!, "w"] .+ rand(Normal(0, noise), nrow(df))
        df[!, "n"] .= df[!, "n"] .+ rand(Normal(0, noise), nrow(df))
        return df
    end
end

In [ ]:
function store_fish_data(w0::Float64,m::Float64,M::Int64,noise::Float64,df::DataFrame)
    CSV.write("c:/Users/lisah/Documents/Repos/ma-code/data/fisher/m0.45/fish_$(w0)_$(m)_$(M)_$(noise).csv", df)
end

In [ ]:
function compute_fish(hprm) # returns trace of fisher information matrix at point pt hprm.n0, hprm.a
    
end

In [ ]:
function gen_all_fish_data(M_vals, noise_vals)
    for M in M_vals
        for noise in noise_vals

            grid = create_grid()
            fish = zeros(41, 21)

            # evaluate fisher info on grid
            for i in range(1, 41)
                for j in range(1, 21)

                    pt = grid[i,j]
                    hprm = Hyperprm(w0, pt[2], pt[1], m, M, noise) #w0,n0,a,m,M

                    fish_val = compute_fish(hprm)
                    fish[i,j] = fish_val
                end
            end
            
            #create data frame
            a_eval_pts = string.(0.0:0.1:2.0)
            df_fish = DataFrame(fish, a_eval_pts)

            store_fish_data(w0, m, M, noise, df_fish)
        end
    end
end

In [ ]:
a_vals = [0.1, 0.9, 1.1, 1.7] 
n0_vals = [0.4, 0.5, 1.0, 1.3, 2.3]

index_combos = [[1,1], [2,4], [3,5], [4,2], [4,3]]

M_vals = [1000]
noise_vals = [0.0]
m = 0.45 # fix for now as given by literature
w0 = 0.95 # mean(a_vals)

gen_all_fish_data(M_vals, noise_vals)